# ocg case study

This case study concerns the transient difference absorption measured in the visible spectral region after 530 nm excitation of an articial light harvester, named orange-calix-green (ocg), in the solvent dichloromethane (dcm). The measurement, the global analysis and the target analysis is based upon the model explained in ([Hippius et al 2008][Hippius2008]).
Below are Questions 2A-E, each worth 10 points.

[Hippius2008]: https://www.few.vu.nl/~ivo/pub/2008/HippiusWilliamsWuerthnerJPCC112_2476_2486.pdf "Hippius C, van Stokkum IHM, Gsanger M, Groeneveld MM, Williams RM, Würthner F (2008) Sequential FRET processes in calix[4]arene-linked orange-red-green perylene bisimide dye zigzag arrays. Journal of Physical Chemistry C 112 (7):2476-2486."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

ocg_dcm_data_path = "Kog_CAa.ascii"
plot_data_overview(ocg_dcm_data_path, nr_of_data_svd_vectors=3, linlog=True, linthresh=1);

## Global Analysis: sequential scheme with decay rates in decreasing order

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

ocg_dcm_global_model_path = "models/ocg_dcm-global-model.yml"
ocg_dcm_global_parameters_path = "models/ocg_dcm-global-parameters.yml"

#### Model file

Note that these data have **not** been preprocessed, thus in contrast to the previous ocg case study an artifact is needed.

In [ ]:
display_file(ocg_dcm_global_model_path, syntax="yaml")

#### Parameters file

The free parameters are the center (the location of the maximum of the IRF at 550 nm, the dispersion_center) and the width of the IRF, and the kinetic parameters.
The dispersion parameters are also estimated from the data.

Question 2A: estimate the missing starting values for the parameters from the data inspection, and insert these in the models/ocg_dcm-global-parameters.yml file. Estimate the parameters, and make sure you achieve a satisfactory fit.

In [ ]:
display_file(ocg_dcm_global_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

ocg_dcm_global_scheme = Scheme(
    model=ocg_dcm_global_model_path,
    parameters=ocg_dcm_global_parameters_path,
    maximum_number_function_evaluations=7,
    data={"ocg_dcm_data": ocg_dcm_data_path},
)
ocg_dcm_global_scheme.validate()

In [ ]:
ocg_dcm_global_result = optimize(ocg_dcm_global_scheme)

In [ ]:
ocg_dcm_global_result.root_mean_square_error

In [ ]:
ocg_dcm_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(ocg_dcm_global_result.data["ocg_dcm_data"])
plot_fitted_traces(
    ocg_dcm_global_result.data["ocg_dcm_data"], wavelengths, linlog=True, linthresh=1
);

Note that in this plot the time axis is linear until 1 ps and logarithmic thereafter. This plot is more informative.

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    ocg_dcm_global_result.data["ocg_dcm_data"],
    linlog=True,
    figure_only=False,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
);

## Dispersion curve

In [ ]:
ocg_dcm_global_result.data["ocg_dcm_data"].data.sel(time=slice(0, 2)).plot()
ocg_dcm_global_result.data["ocg_dcm_data"].irf_center_location.plot()

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(ocg_dcm_global_result)

Question 2B: Comment on the interpretation of the EADS (relating each EADS to the estimated lifetimes), with the help of abs_emis_org.png (also copied in the ocg-case-study folder) and other course materials. Discuss especially the different contributions to the normalized third and the fourth EADS. Hint 1: to insert an asterisk to indicate an excited state, it must be preceded by a backslash, e.g. o\*. Hint 2: Explain the shape of the normalized fourth EADS between 450 and 600 nm.

Answer 2B: 

##Refined Target Analysis

### Used model and parameters

In [ ]:
ocg_dcm_target_model_path = "models/ocg_dcm-refined-target-model.yml"
ocg_dcm_target_parameters_path = "models/ocg_dcm-refined-target-parameters.yml"

#### Model file
This refined target analysis concentrates on resolving the different contributions to the third and the fourth EADS. It employs additional spectral information of the type "clp-guide", see the names of the three extra datasets.

In [ ]:
display_file(ocg_dcm_target_model_path, syntax="yaml")

#### Parameters file

Question 2C: Study the kinetic scheme ocg_dcm_scheme.png (in the ocg-case-study folder) and the model file carefully.
Insert the missing starting values in the models/ocg_dcm-target-parameters.yml file. Hint: use the above estimated kinetic and IRF parameters. Study the parameters file carefully. Discuss the input in relation to abs_emis_org.png (also copied in the ocg-refined-case-study folder)

Answer 2C:

In [ ]:
display_file(ocg_dcm_target_parameters_path, syntax="yaml")

### Create scheme and optimize it
Note that the # sign in front of import has to be removed

In [ ]:
# import glotaran.builtin.megacomplexes.clp_guide

ocg_dcm_target_scheme = Scheme(
    model=ocg_dcm_target_model_path,
    parameters=ocg_dcm_target_parameters_path,
    maximum_number_function_evaluations=4,
    data={
        "ocg_dcm_data": ocg_dcm_data_path,
        "tripletSADS": "tripletSADSblue.ascii",
        "RPSADS": "RPSADS.ascii",
        "cg_dcmSADS": "4cg_dcmSADSblue.ascii",
    },
)
ocg_dcm_target_scheme.validate()

In [ ]:
ocg_dcm_target_result = optimize(ocg_dcm_target_scheme)

In [ ]:
ocg_dcm_target_result.root_mean_square_error

In [ ]:
ocg_dcm_target_result

In [ ]:
ocg_dcm_target_result.optimized_parameters

In [ ]:
compartments = ocg_dcm_target_scheme.model.initial_concentration["input1"].compartments

ocg_dcm_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

### Result plots

#### Fit quality

In [ ]:
fig_target, axes = plot_fitted_traces(
    ocg_dcm_target_result.data["ocg_dcm_data"], wavelengths, linlog=True, linthresh=1
);

#### Comparison of the estimated SADS (orange) and the guidance spectra (blue) 

In [ ]:
ocg_dcm_target_result.data["tripletSADS"].data.plot()
ocg_dcm_target_result.data["tripletSADS"].fitted_data.plot()

In [ ]:
ocg_dcm_target_result.data["cg_dcmSADS"].data.plot()
ocg_dcm_target_result.data["cg_dcmSADS"].fitted_data.plot()

In [ ]:
ocg_dcm_target_result.data["RPSADS"].data.plot()
ocg_dcm_target_result.data["RPSADS"].fitted_data.plot()

#### Overview

In [ ]:
plot_overview(
    ocg_dcm_target_result.data["ocg_dcm_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

Question 2D (to be answered in combination with part E): Compare the rmse of the global and of the target analysis. How large is the yield (in %) of g\* relative to the first excited state? How large is the yield (in %) of the fifth species relative to the first excited state? Comment on the interpretation of the SADS (i.e. give each species a name that refers to the properties of the state), with the help of abs_emis_org.png (also copied in the ocg-case-study folder) and other course materials. Discuss the differences between the black, red and blue SADS (referring to the Table that you fill in part E). Discuss the differences between the normalized fourth EADS and the normalized fourth SADS (green), especially between 450 and 600 nm. Interpret the fifth SADS (magenta) and assign it to a chromophore.

Answer 2D: 

Question 2E: In the Table below fill the **names** from part D, and fill at each ? the wavelength of the main band in the SADS from the above target analysis

| Species     | color | Bleach (nm)  | SE (nm) | absorption (nm) |
| ----------- | ----- | ------------ | ------- | --------------- |
| name        | black | ?,?     | ?,? | ?             |
| name        | red   | ?,?     | ?,? | ?             |
| name        | blue  | ?,?     |         | ?             |
| name        | green | ?,?,? | ?     | ?,?         |
| name         | magenta | ?,? |      | ?         |